# MEP - Trabalho

## Resumo

Blablabla

## Introdução

Blablabla

## Banco de dados

Vamos utilizar o banco de dados da SCOPUS([1](https://service.elsevier.com/app/answers/detail/a_id/15181/supporthub/scopus/)). Neste banco de dados, cada revista é classificada em uma área por especialistas humanos. Os artigos de cada revistam herdam a sua classificação de área. No momento de escrita deste artigo, a grande área de *Physical Sciences* contava com 115 subáreas. Para fins de exequibilidade, nos restringimos às 13 áreas relacionadas à Ciência da Computação e às 15 áreas relacionadas à Matemática:

- 1700 - General Computer Science
- 1701 - Computer Science (miscellaneous)
- 1702 - Artificial Intelligence
- 1703 - Computational Theory and Mathematics
- 1704 - Computer Graphics and Computer-Aided Design
- 1705 - Computer Networks and Communications
- 1706 - Computer Science Applications
- 1707 - Computer Vision and Pattern Recognition
- 1708 - Hardware and Architecture
- 1709 - Human-Computer Interaction
- 1710 - Information Systems
- 1711 - Signal Processing
- 1712 - Software

Por considerações de tempo, consideramos somente os tópicos de 1702 a 1706. A pesquisa avançada do próprio SCOPUS([2](https://www.scopus.com/search/form.uri?display=basic#basic)) nos permitiu obter resultados dentro de cada uma dessas classificações, excluindo-se aqueles que aparecem com mais de uma classificação, com strings de busca como

    SUBJTERMS ( 1702 ) AND NOT ( SUBJTERMS ( 1703 ) OR SUBJTERMS ( 1704 ) OR SUBJTERMS ( 1705 ) OR SUBJTERMS ( 1706 ) )
    
O único metadado com o qual nos preocupamos são as palavras-chave.

Consideramos os artigos com os maiores números de citações, que presumimos formarem bons representantes de diversas áreas do conhecimento (conforme a classificação humana). Uma alternativa seria considerar artigos conforme o parâmetro "relevância" da busca do SCOPUS, mas este parece sempre retornar artigos mais recentes.

 Nos restringimos a analisar somente as 2000 primeiras entradas de cada uma das áreas sob consideração. Excluindo entradas nas quais faltam informações, obtivemos aproximadamente 1000 entradas de cada área.

In [1]:
import pandas
import numpy as np

In [2]:
dados = []
for i in range(5):
    dados.append(pandas.read_csv(f"170{i+2}.csv").drop(labels="Link",axis=1).dropna().reset_index(drop=True))

Tendo nosso banco de dados em mãos, precisamos determinar um processo de análise de similaridade de entre palavras-chave e entre conjuntos dessas. Porém, também é comum que palavras-chave sejam compostas por mais de uma palavra, separadas por hífens e/ou espaços. Neste tipo de situação, separamos todas as palavras-chave compostas em suas constituintes. Também omitimos palavras com dois ou menos caracteres, 

In [3]:
for i in range(5):
    dados[i]["Author Keywords"] = (dados[i]["Author Keywords"].str.lower()).str.split(r"\ |/|;|-|\\|,",regex=True).apply(lambda l : [x for x in l if len(x)>3])

Deste modo, cada trabalho é identificado como o seu conjunto de palavras-chave. Nosso principal objetivo, agora, é elaborar uma métrica de similaridade entre conjuntos de palavras.

Existem diversas técnicas para determinar similaridade entre palavras individuais. Técnicas sintáticas, como aquelas envolvendo Distância de Levenshtein e Similaridade de N-gramas, não são apropriadas para nossos fins, pois nosso foco é o conteúdo dos trabalhos. Neste trabalho, adotamos um modelo pré-treinado de Word2Vec disponível na biblioteca de Python Gensim. O modelo que escolhemos é o "word2vec-google-news-300", que é treinado em parte do conjunto de notícias do Google News, e conta com aproximadamente 3 milhões de frases e palavras. Este modelo permite que representemos palavras como vetores num espaço euclidiano 300-dimensional, e foi escolhido por sua grande abrangência.


In [4]:
# Para baixar o modelo pela primeira vez, utilize as próximas 2 linhas
# import gensim.downloader as api
# modelow2v = api.load('word2vec-google-news-300')
#
# Para salvar o modelo, utilize a próxima linha
# modelow2v.save("gn300")
#
# O modelo tem mais de ~1662.8MB, então vale a pena o salvar em disco para acesso mais rápido

from gensim.models import KeyedVectors

In [5]:
import gensim.downloader as api
modelow2v = api.load('word2vec-google-news-300')

In [6]:
# modelow2v = KeyedVectors.load("gn300", mmap='r')

O próximo problema a ser abordado é "Como utilizar uma métrica de semelhança de palavras para determinar uma métrica de semelhança de conjuntos de palavras?". Os conjuntos de palavras-chave não consistem de sentenças estruturadas com significado preciso; são apenas coleções de palavras que servem para resumir o assunto geral que é abordado no artigo. Por isso, modelos de assemelhação semântica de frases não são apropriados no nosso contexto.

Este é um problema que não tem uma resposta bem-estabelecida, inclusive por seu escopo limitado (https://link.springer.com/chapter/10.1007/978-3-662-44415-3_20). Por isso, aplicamos uma abordagem genérica simples, nos utilizando do fato de que as palavras-chave individuais já estão representadas numericamente: Cada documento será representado, num espaço euclidiano 300-dimensional, pela média (normalizada sobre a esfera) de suas palavras-chave.

In [7]:
trabalhos_vetorizados = []

for i in range(5):
    trabalhos_vetorizados.append([])
    for j in range(len(dados[i])):
        trabalho_vetorizado = np.zeros(300)
        for word in dados[i]["Author Keywords"][j]:
            try:
                trabalho_vetorizado += np.array(modelow2v[word])/np.linalg.norm(np.array(modelow2v[word]))
            except:
                pass
        
        if np.linalg.norm(trabalho_vetorizado)==0:
            # Um trabalho não pode ser vetorizado
            pass
        else:
            trabalho_vetorizado = trabalho_vetorizado/np.linalg.norm(trabalho_vetorizado)
            trabalhos_vetorizados[i].append(trabalho_vetorizado)
        
todos_trabalhos_vetorizados = []
for i in range(5):
    todos_trabalhos_vetorizados += trabalhos_vetorizados[i]


Isto permitirá aplicar algoritmos de clusterização sem necessidade de retrabalho. Para tal, escolhemos o algoritmo $K$-means clustering, disponível na biblioteca scikitlearn. Para completude, utilizamos outros métodos disponíveis nesta biblioteca.

In [8]:
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, BisectingKMeans

kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto", max_iter = 300).fit(todos_trabalhos_vetorizados)
spcluster = SpectralClustering(n_clusters = 5).fit(todos_trabalhos_vetorizados)
agcluster = AgglomerativeClustering(n_clusters = 5).fit(todos_trabalhos_vetorizados)
bkmeans = BisectingKMeans(n_clusters = 5).fit(todos_trabalhos_vetorizados)

In [9]:
# Cria os clusteres após o treinamento
clusteres_kmeans = []
for i in range(5):
    clusteres_kmeans.append([])
for i in range(len(todos_trabalhos_vetorizados)):
    clusteres_kmeans[kmeans.labels_[i]].append(todos_trabalhos_vetorizados[i])

for i in range(5):
    print(f"O cluster {i} tem {len(clusteres_kmeans[i])} trabalhos")

O cluster 0 tem 855 trabalhos
O cluster 1 tem 1158 trabalhos
O cluster 2 tem 1175 trabalhos
O cluster 3 tem 1186 trabalhos
O cluster 4 tem 1429 trabalhos


Por fim, calculamos os Índices de Davies-Bouldin padrão do clustering após o treinamento via $K$-means, e o comparamos com o índice do clustering antes do treinamento.

In [10]:
# Denecessário

def DB(clusteres , centroides = None):

    n = len(clusteres)

    if centroides is None:
        centroides = []
        for i in range(n):
            # Calcula o centroide
            centroide = np.sum(clusteres[i],axis=0)/len(clusteres[i])
            centroides.append(centroide)


    S = np.zeros(n)
    for i in range(n):
        centroide = centroides[i]
        S[i] = 1/len(clusteres[i]) * np.sum([np.linalg.norm(trabalho - centroide) for trabalho in clusteres[i]])

    M = np.zeros((n,n))
    R = np.zeros((n,n))
    D = np.zeros(n)
    for i in range(n):
        for j in range(n):
            if i!= j:
                M[i,j]= np.linalg.norm(centroides[i]-centroides[j])
                R[i,j] = (S[i]+S[j])/M[i,j]
                D[i] = max(D[i],R[i,j])
    return (1/n)*np.sum(D)

In [11]:
from sklearn.metrics import davies_bouldin_score

In [12]:
print(f"O índice de Davies-Bouldin da clusterização original é {davies_bouldin_score(todos_trabalhos_vetorizados,[0]*len(trabalhos_vetorizados[0])+[1]*len(trabalhos_vetorizados[1])+[2]*len(trabalhos_vetorizados[2])+[3]*len(trabalhos_vetorizados[3])+[4]*len(trabalhos_vetorizados[4]))}")

O índice de Davies-Bouldin da clusterização original é 9.319913208305726


In [13]:
print(f"O índice de Davies-Bouldin da nova clusterização por K-Means é {davies_bouldin_score(todos_trabalhos_vetorizados,kmeans.labels_)}")

O índice de Davies-Bouldin da nova clusterização por K-Means é 4.470097311715916


In [14]:
print(f"O índice de Davies-Bouldin da nova clusterização por Spectral Clustering é {davies_bouldin_score(todos_trabalhos_vetorizados,spcluster.labels_)}")

O índice de Davies-Bouldin da nova clusterização por Spectral Clustering é 4.423220794339837


In [15]:
print(f"O índice de Davies-Bouldin da nova clusterização por Agglomerative Clustering é {davies_bouldin_score(todos_trabalhos_vetorizados,agcluster.labels_)}")

O índice de Davies-Bouldin da nova clusterização por Agglomerative Clustering é 5.454415182346748


In [16]:
print(f"O índice de Davies-Bouldin da nova clusterização por Bisecting K-Means é {davies_bouldin_score(todos_trabalhos_vetorizados,bkmeans.labels_)}")

O índice de Davies-Bouldin da nova clusterização por Bisecting K-Means é 5.016578981587873


O menor índice de Davies-Bouldin é o da nova clusterização por K-Means, o que indica que ela é melhor.

# Considerações finais

Utilizando técnicas bem-estabelecidas e facilmente acessíveis de aprendizado de máquinas, pudemos aprimorar - quantitativamente - a classificação humana sobre 5 subáreas do conhecimento de Ciência da Computação. Isso mostra que essa é uma direção promissora de trabalhos. Porém, trabalhamos em um ambiente bastante limitado

Existem outras técnicas de análise semântica que poderiam ser adotadas (LSA sendo uma bem-conhecida), porém a praticidade de implementação de modelos de Word2Vec a fez preferível neste trabalho.

Modelos baseados nas notícias do Google podem diferir significativamente de modelos treinados em grande bancos de trabalhos acadêmicos. Porém, não conseguimos encontrar modelos grande baseados em textos acadêmicos.

Similarmente, o método de comparação de clusterings via índice de Davies-Boulding foi escolhido por ser apropriado ao modo com que trabalhamos com os dados.

Por fim, a própria atribuição de palavras-chave aos trabalhos científicos pelos seus autores pode gerar o mesmo tipo de erro que o esperado nas classificações humanas dos campos de estudo de revistas científicas e artigos, que buscamos aprimorar. Uma análise mais efetiva poderia levar em consideração os textos completos. O acesso aos textos completos, por sua vez, é um fator impeditivo para tal análise, visto que as maiores editoras requerem assinaturas por parte dos leitores ou pagamento de taxas de *open access* por parte dos autores, ambas historicamente com valores proibitivos. Bases de dados abertas e amplamente utilizadas em certas linhas de pesquisa, tais como o [arXiv](arxiv.org), seriam uma poderosa ferramenta alternativa para possibilitar esse tipo de análise.

Referências adicionais (não necessariamente botar na lista de referências!!)

https://stackoverflow.com/questions/21979970/how-to-use-word2vec-to-calculate-the-similarity-distance-by-giving-2-words

https://en.wikipedia.org/wiki/K-means_clustering
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://stackoverflow.com/questions/21979970/how-to-use-word2vec-to-calculate-the-similarity-distance-by-giving-2-words

https://radimrehurek.com/gensim_3.8.3/sklearn_api/w2vmodel.html

https://scikit-learn.org/stable/tutorial/basic/tutorial.html

filegpt